In [11]:
import pandas as pd
import strats
from backtesting import Backtest, Strategy

In [2]:
logsignal_df = pd.read_csv("lr_signal_data/{}_lr_signal.csv".format("SPY"), index_col="Date", parse_dates=True)
logsignal_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,LR_Signal
Date,,,,,,,,
2014-03-24,163.208719,163.409639,161.269501,161.977051,121411000,0.000,0,0
2014-03-25,162.798167,163.296081,161.837302,162.745758,103852000,0.000,0,0
2014-03-26,163.383485,163.645544,161.531623,161.575302,119843000,0.000,0,1
2014-03-27,161.383116,161.898491,160.640618,161.234619,142383000,0.000,0,1
2014-03-28,161.697538,162.841848,161.601450,162.029480,101642000,0.000,0,0
...,...,...,...,...,...,...,...,...
2021-03-18,393.192202,395.424880,389.474368,390.201996,115112500,0.000,0,1
2021-03-19,389.880005,391.570007,387.149994,389.480011,112777800,1.278,0,0
2021-03-22,390.029999,394.070007,389.970001,392.589996,73778600,0.000,0,0


In [8]:
period = 2015
if isinstance(period, str):
    logsignal_data = logsignal_df.iloc[-int(float(period)*252):]
elif isinstance(period, int):
    logsignal_data = logsignal_df.loc["{}-12-31".format(period-1):"{}-12-31".format(period),]
logsignal_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,LR_Signal
Date,,,,,,,,
2014-12-31,184.419943,184.597276,182.114578,182.247574,130333800,0.0,0,0
2015-01-02,182.992332,183.435671,181.041633,182.149979,121465900,0.0,0,0
2015-01-05,181.032799,181.210131,178.532377,178.860443,169632600,0.0,0,1
2015-01-06,179.188528,179.747139,176.324565,177.175781,209151400,0.0,0,1
2015-01-07,178.594449,179.747131,178.115649,179.383591,125346700,0.0,0,0
...,...,...,...,...,...,...,...,...
2015-12-24,186.171201,186.723236,185.899706,186.134995,48539600,0.0,0,0
2015-12-28,185.392940,185.754924,184.560366,185.709686,65899900,0.0,0,0
2015-12-29,186.886118,188.044483,186.849925,187.691544,92640700,0.0,0,0


In [9]:
logsignal_data.LR_Signal.unique()

array([ 0,  1, -1])

In [12]:
class LogReg_Signal(Strategy): 
    # Define parameters of the strategy
    buy_signal = 1
    sell_signal = -1
    long_only = 1
    
    def init(self):
        # Compute signal
        self.lr_sig = self.data.LR_Signal
        
    def next(self):
        # If RSI enters oversold territory
        if self.lr_sig == self.buy_signal:
            if self.long_only == 0:
                self.position.close()
            self.buy()
        
        # If RSI enters overbought territory
        elif self.lr_sig == self.sell_signal:
            self.position.close()
            if self.long_only == 0:
                self.sell()

In [14]:
bt = Backtest(logsignal_data, strats.LogReg_Signal, cash=100000, commission=0)
stats = bt.run()
stats

Start                     2014-03-24 00:00:00
End                       2021-03-24 00:00:00
Duration                   2557 days 00:00:00
Exposure Time [%]                     99.8866
Equity Final [$]                       237172
Equity Peak [$]                        242339
Return [%]                            137.172
Buy & Hold Return [%]                 139.244
Return (Ann.) [%]                     13.1307
Volatility (Ann.) [%]                 20.0503
Sharpe Ratio                         0.654885
Sortino Ratio                         1.03038
Calmar Ratio                         0.389453
Max. Drawdown [%]                    -33.7157
Avg. Drawdown [%]                    -1.56045
Max. Drawdown Duration      273 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    1
Win Rate [%]                              100
Best Trade [%]                        139.314
Worst Trade [%]                       139.314
Avg. Trade [%]                    